<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys  # Provides access to system-specific parameters and functions, such as manipulating the Python runtime environment or handling command-line arguments.

import requests  # Simplifies making HTTP requests in Python (like GET and POST). Commonly used to interact with APIs or download web content.

from bs4 import BeautifulSoup  # A library used for parsing HTML and XML documents. It's helpful in web scraping for extracting data from web pages.

import re  # Provides support for working with regular expressions in Python, useful for searching and manipulating strings based on specific patterns.

import unicodedata  # Offers functions to work with Unicode data. It helps normalize, inspect, or manipulate Unicode text.

import pandas as pd  # A powerful data manipulation and analysis library. It provides data structures like DataFrames, ideal for handling and analyzing structured data.

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
data = requests.get(static_url).text

Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(data, "html.parser")

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables=soup.find_all('tr')

Starting from the third table is our target table contains the actual launch records.


In [9]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [10]:
column_names = []

# Assuming first_launch_table contains the HTML table and extract_column_from_header() is defined
# Apply find_all() to get all <th> elements from the table
th_elements = first_launch_table.find_all('th')

# Iterate through each <th> element
for th in th_elements:
    # Extract column name using the provided extract_column_from_header function
    column_name = extract_column_from_header(th)
    
    # Check if the column name is not None and has a length greater than 0
    if column_name is not None and len(column_name) > 0:
        # Append the column name to the column_names list
        column_names.append(column_name)

# Now column_names contains the list of non-empty column names

Check the extracted column names


In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [14]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [16]:
extracted_row = 0
launch_dict = {}  # Initialize the dictionary to store launch data

# Extract each table
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table rows
    for rows in table.find_all("tr"):
        # Check if the first table heading (th) is a number corresponding to launch number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()  # Check if flight_number is a digit
        else:
            flag = False
        
        # Get table elements (td cells)
        row = rows.find_all('td')
        
        # If the flight number is valid, process the row
        if flag:
            extracted_row += 1
            
            # Append the flight_number into launch_dict with key `Flight No.`
            launch_dict[extracted_row] = {'Flight No.': flight_number}
            
            # Date and Time
            datatimelist = date_time(row[0])
            
            # Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict[extracted_row]['Date'] = date
            
            # Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict[extracted_row]['Time'] = time
            
            # Booster version
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string
            launch_dict[extracted_row]['Version Booster'] = bv
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict[extracted_row]['Launch Site'] = launch_site
            
            # Payload
            payload = row[3].a.string
            launch_dict[extracted_row]['Payload'] = payload
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict[extracted_row]['Payload mass'] = payload_mass
            
            # Orbit
            orbit = row[5].a.string
            launch_dict[extracted_row]['Orbit'] = orbit
            
            # Customer
            if row[6].a:  # Check if <a> tag exists
                customer = row[6].a.string
            else:
                customer = None  # Handle the case where there is no customer data
            launch_dict[extracted_row]['Customer'] = customer

            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict[extracted_row]['Launch outcome'] = launch_outcome
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict[extracted_row]['Booster landing'] = booster_landing


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [17]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [18]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,112,113,114,115,116,117,118,119,120,121
Flight No.,1,2,3,4,5,6,7,8,9,10,...,112,113,114,115,116,117,118,119,120,121
Date,4 June 2010,8 December 2010,22 May 2012,8 October 2012,1 March 2013,29 September 2013,3 December 2013,6 January 2014,18 April 2014,14 July 2014,...,24 March 2021,7 April 2021,23 April 2021,29 April 2021,4 May 2021,9 May 2021,15 May 2021,26 May 2021,3 June 2021,6 June 2021
Time,18:45,15:43,07:44,00:35,15:10,16:00,22:41,22:06,19:25,15:15,...,08:28,16:34,9:49,03:44,19:01,06:42,22:56,18:59,17:29,04:26
Version Booster,F9 v1.07B0003.18,F9 v1.07B0004.18,F9 v1.07B0005.18,F9 v1.07B0006.18,F9 v1.07B0007.18,F9 v1.17B10038,F9 v1.1,F9 v1.1,F9 v1.1,F9 v1.1,...,F9 B5B1060.6643,F9 B5 ♺,F9 B5B1061.2647,F9 B5B1060.7652,F9 B5B1049.9655,F9 B5B1051.10657,F9 B5B1058.8660,F9 B5B1063.2665,F9 B5B1067.1668,F9 B5
Launch Site,CCAFS,CCAFS,CCAFS,CCAFS,CCAFS,VAFB,CCAFS,CCAFS,Cape Canaveral,Cape Canaveral,...,CCSFS,CCSFS,KSC,CCSFS,KSC,CCSFS,KSC,CCSFS,KSC,CCSFS


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
